# Importing Libraries

In [ ]:
import os
import time
import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm_notebook as tqdm
import random
import glob

import cv2
from PIL import Image
from keras.layers import Input, Dense, BatchNormalization, Conv2D, Conv2DTranspose, ReLU, LeakyReLU, Flatten, MaxPooling2D, Dropout, Reshape

# Preprocessing (Resizing and Normalization)

# Resizing images 224x224(You can run once to create a dataset of 224x224 resized images)

In [ ]:
from PIL import Image  # For image loading and processing
import os              # For directory and file path operations

# Input and Output directories
input_dir = '/kaggle/input/dfuc2021/dataset/PartA_DFU_dataset/PartA_DFU_Dataset'  # Original dataset path
output_dir = '/kaggle/working/resized_dataset'  # Destination to save resized images

# Class subfolders (assuming binary classification: Normal vs Abnormal)
folders = ['Normal', 'Abnormal']

# Target size for resizing all images
target_size = (224, 224)

# Iterate over both folders (classes)
for folder in folders:
    input_folder = os.path.join(input_dir, folder)       # e.g., /kaggle/input/.../Normal
    output_folder = os.path.join(output_dir, folder)     # e.g., /kaggle/working/.../Normal
    os.makedirs(output_folder, exist_ok=True)            # Create output folder if it doesn't exist

    # Loop over each file in the class folder
    for filename in os.listdir(input_folder):
        # Process only common image formats
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            try:
                # Open image and ensure it's in RGB format
                img = Image.open(input_path).convert('RGB')
                
                # Resize image to target dimensions using high-quality resampling
                img = img.resize(target_size, Image.Resampling.LANCZOS)  # LANCZOS is preferred for quality

                # Save the resized image to the output path
                img.save(output_path)

            except Exception as e:
                # Catch and report any errors during processing
                print(f"Error processing {input_path}: {e}")


# No. of images in each class

In [ ]:
from PIL import Image
import os

# Set the directory to check
base_dir = '/kaggle/input/dfuc-parta-resized'  # or wherever your images are saved
folders = ['Normal', 'Abnormal']

# Iterate through both folders
for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    print(f"\n--- Image sizes in folder: {folder} ---")
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            path = os.path.join(folder_path, filename)
            try:
                img = Image.open(path)
                print(f"{filename}: {img.size}")  # prints size like (224, 224)
            except Exception as e:
                print(f"Error with {filename}: {e}")


# Download the resized_dataset

In [ ]:
import shutil
shutil.make_archive("/kaggle/working/resized_dataset", 'zip', "/kaggle/working/resized_dataset")

# Sample Images Abnormal

In [ ]:
# Define the path to the dataset
PATH1 = '/kaggle/input/dfuc-parta-resized/Abnormal'

# List all images in the folder
images = os.listdir(PATH1)
print(f'There are {len(images)} pictures of Abnormal.')

# Create a grid of 3x3 images
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 10))

for indx, axis in enumerate(axes.flatten()):
    # Randomly select an image
    rnd_indx = np.random.randint(0, len(images))
    img_path = os.path.join(PATH1, images[rnd_indx])  # Construct the correct file path
    img = plt.imread(img_path)  # Read the image
    imgplot = axis.imshow(img)  # Display the image
    axis.set_title(images[rnd_indx])  # Set the title to the image filename
    axis.set_axis_off()  # Turn off axis ticks and labels

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Sample images Normal

In [ ]:
# Define the path to the dataset
PATH1 = '/kaggle/input/dfuc-parta-resized/Normal'

# List all images in the folder
images = os.listdir(PATH1)
print(f'There are {len(images)} pictures of Normal.')

# Create a grid of 3x3 images
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 10))

for indx, axis in enumerate(axes.flatten()):
    # Randomly select an image
    rnd_indx = np.random.randint(0, len(images))
    img_path = os.path.join(PATH1, images[rnd_indx])  # Construct the correct file path
    img = plt.imread(img_path)  # Read the image
    imgplot = axis.imshow(img)  # Display the image
    axis.set_title(images[rnd_indx])  # Set the title to the image filename
    axis.set_axis_off()  # Turn off axis ticks and labels

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Runtime Configurations

In [ ]:
TRAIN_PATH = '/kaggle/input/dfuc-parta-resized'

BATCH_SIZE = 32
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
NUM_CLASSES = 2
EPOCHS = 200

# Visualization of images from different folders

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

# Path to your dataset (containing two subfolders: Normal/ and Abnormal/)
data_dir = "/kaggle/input/dfuc-parta-resized"  # Change to your dataset path

# Load dataset without any transform
train_data = ImageFolder(root=data_dir)

# Get class names (folder names)
class_names = train_data.classes

# Randomly select 10 image indices
indices = random.sample(range(len(train_data)), 10)

# Plot
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i, idx in enumerate(indices):
    path, label = train_data.samples[idx]  # This gives image path and label
    img = Image.open(path).convert("RGB")  # Open and ensure 3 channels

    axes[i].imshow(img)
    axes[i].set_title(f"Label: {class_names[label]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Print class name details
print("Class Names:", class_names)
print("Index of 'Abnormal':", class_names.index("Abnormal"))
print("Index of 'Normal':", class_names.index("Normal"))


# Visualizing Data Imbalance

In [ ]:
# Import required libraries
import tensorflow as tf                  
import matplotlib.pyplot as plt         
import numpy as np                    

# Define the directory path to the dataset
TRAIN_PATH = '/kaggle/input/dfuc-parta-resized'

# Automatically infers labels from subfolder names
# Loads images in batches of 32, resizes them to specified dimensions
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',                   # Labels are inferred from folder names
    batch_size=32,                       # Number of images per batch
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # Resize all images to this size
    seed=0                               # Seed for consistent shuffling
)

# Initialize an array to store class counts: index 0 for 'Abnormal', index 1 for 'Normal'
total_counts = np.array([0, 0])  # Format: [count of class 0, count of class 1]

# Loop through all batches in the dataset
for _, labels in full_ds:
    # Get unique class labels and their counts in this batch
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    
    # Add batch counts to the running total for each class
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names (must match folder names or their inferred order)
class_names = ["Abnormal", "Normal"]

# Define bar colors for each class: red and blue
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal

# Create a bar plot to show number of samples per class
plt.figure(figsize=(8, 6))                   # Set figure size
plt.bar(class_names, total_counts, color=colors)  # Plot the class counts
plt.xlabel("Class")                         # X-axis label
plt.ylabel("Number of Samples")             # Y-axis label
plt.title("Class Distribution in Original Dataset")  # Plot title
plt.tight_layout()                          # Adjust layout to prevent clipping
plt.show()                                  # Display the plot

# Print the total count of images in each class
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")

# Weights Initialization

In [ ]:
def weights_init(m):
    """
    Takes as input a neural network m that will initialize all its weights.
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)  #Stabilizes early training, avoids large weights
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)   #Keeps initial scaling/shift minimal
        m.bias.data.fill_(0)

# Data Augmentation using Enhanced Conditional GAN (Training)

In [ ]:
import torch 
import torch.nn as nn  # Neural network components
import torch.optim as optim  
from torch.utils.data import DataLoader  
from torchvision import datasets, transforms  
import os
import matplotlib.pyplot as plt

# Choose GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------------------------------
# Generator Definition
# --------------------------------
class Generator(nn.Module):
    def __init__(self, nz=128, channels=3, num_classes=2):
        super(Generator, self).__init__()
        self.nz = nz
        self.label_embedding = nn.Embedding(num_classes, nz)

        def block(in_feat, out_feat, k_size=4, stride=2, padding=1, dropout=False):
            layers = [
                nn.ConvTranspose2d(in_feat, out_feat, k_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_feat),
                nn.ReLU(True)
            ]
            if dropout:
                layers.append(nn.Dropout(0.3))
            return layers

        self.model = nn.Sequential(
            *block(nz * 2, 1024, 4, 1, 0),   # Input channels = nz*2=256 (noise + label), output 1024 channels; kernel=4, stride=1, padding=0; output size: 1x1 → 4x4 feature map # 1x1 → 4x4
            *block(1024, 512),              # 4x4 → 8x8
            *block(512, 256),               # 8x8 → 16x16
            *block(256, 128),               # 16x16 → 32x32
            *block(128, 64),                # 32x32 → 64x64
            *block(64, 32),                 # 64x64 → 128x128
            *block(32, 16),                 # 128x128 → 256x256
            nn.ConvTranspose2d(16, channels, 4, 2, 1),  # 256x256 → 512x512
            nn.Upsample(size=(224, 224)),              # Resize final output to 224×224
            nn.Tanh()
        )

    def forward(self, z, labels):
        embed = self.label_embedding(labels)
        x = torch.cat((z, embed), dim=1).view(-1, self.nz * 2, 1, 1)
        return self.model(x)
# --------------------------------
# Discriminator Definition
# --------------------------------
class Discriminator(nn.Module):
    def __init__(self, channels=3, num_classes=2, img_size=224):
        super(Discriminator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, img_size * img_size)  # Embed label as image shape

        def block(in_feat, out_feat, k_size=4, stride=2, padding=1, bn=False, dropout=False):
            layers = [nn.Conv2d(in_feat, out_feat, k_size, stride, padding, bias=False)]
            if bn:
                layers.append(nn.BatchNorm2d(out_feat))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            if dropout:
                layers.append(nn.Dropout(0.3))
            return layers

        self.model = nn.Sequential(
            *block(channels + 1, 64, dropout=True),       # Conv2d: input channels + 1 (for label), output 64; dropout applied; spatial downsample by 2 (default stride=2)
            *block(64, 128, bn=True, dropout=True),       # Conv2d: 64 → 128 channels; batch norm + dropout; downsample spatially by 2
            *block(128, 256, bn=True, dropout=True),      # Conv2d: 128 → 256 channels; batch norm + dropout; downsample spatially by 2
            *block(256, 512, bn=True, dropout=True),      # Conv2d: 256 → 512 channels; batch norm + dropout; downsample spatially by 2
            nn.Conv2d(512, 1, 4, 1, 0),                   # Conv2d: reduce channels to 1 with kernel 4, stride 1, no padding; reduces spatial size further
            nn.AdaptiveAvgPool2d(1),                       # Adaptive average pool to make output spatial size 1x1 (scalar per batch)
            nn.Sigmoid()    
        )

    def forward(self, img, labels):
        embed = self.label_embedding(labels).view(-1, 1, 224, 224)  # Match image size
        x = torch.cat((img, embed), dim=1)  # Concatenate label as extra channel
        return self.model(x).view(-1, 1)    # Output shape: [B, 1]



# --------------------------------
# Weight Initialization Function
# --------------------------------
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1 or classname.find("Linear") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)  # Initialize Conv/Linear weights
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)  # Initialize BatchNorm weights
        nn.init.constant_(m.bias.data, 0)  # Initialize bias

# --------------------------------
# Hyperparameters and Model Setup
# --------------------------------
EPOCHS = 200
LR = 0.0002
BATCH_SIZE = 32
NZ = 128  # Noise dimension
PATIENCE = 10
IMAGE_SIZE = 224
CHANNELS = 3
NUM_CLASSES = 2

# Instantiate models
netG = Generator(nz=NZ, channels=CHANNELS, num_classes=NUM_CLASSES).to(device)
netD = Discriminator(channels=CHANNELS, num_classes=NUM_CLASSES).to(device)

# Apply custom weight initialization
netG.apply(weights_init)
netD.apply(weights_init)

# Optimizers
optimizerG = optim.Adam(netG.parameters(), lr=LR, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=LR, betas=(0.5, 0.999))

# Loss function: Binary cross-entropy for real vs fake classification
criterion = nn.BCELoss()

# --------------------------------
# Dataset and DataLoader
# --------------------------------
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5,) * 3, (0.5,) * 3)  # Normalize to [-1, 1] for Tanh compatibility
])

# Load training data
train_data = datasets.ImageFolder('/kaggle/input/dfuc-parta-resized', transform=transform)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

# Fixed noise and labels for evaluating generator progress during training
fixed_noise = torch.randn(16, NZ, device=device)
fixed_labels = torch.full((16,), 1, dtype=torch.long, device=device)

# --------------------------------
# Training Loop
# --------------------------------
G_losses, D_losses, D_accuracies = [], [], []  # Store loss and accuracy metrics
best_loss = float('inf')
counter = 0

for epoch in range(EPOCHS):
    netG.train(); netD.train()
    epoch_g_loss, epoch_d_loss, correct = 0, 0, 0
    total = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.size(0)

        # Ground truth labels
        real = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)

        # ======== Train Discriminator ========
        optimizerD.zero_grad()

        # Discriminator loss on real images
        out_real = netD(imgs, labels)
        loss_real = criterion(out_real, real)

        # Generate fake images
        z = torch.randn(batch_size, NZ, device=device)
        fake_labels = torch.randint(0, NUM_CLASSES, (batch_size,), device=device)
        fake_imgs = netG(z, fake_labels)

        # Discriminator loss on fake images
        out_fake = netD(fake_imgs.detach(), fake_labels)
        loss_fake = criterion(out_fake, fake)

        d_loss = loss_real + loss_fake
        d_loss.backward()     #backpropagation of discriminator loss
        optimizerD.step()

        # ======== Train Generator ========
        optimizerG.zero_grad()
        out_fake_for_g = netD(fake_imgs, fake_labels)
        g_loss = criterion(out_fake_for_g, real)  # Generator tries to trick discriminator
        g_loss.backward()     #backpropagation of generator loss
        optimizerG.step()

        # Accuracy of Discriminator
        pred_real = (out_real > 0.5).float()
        pred_fake = (out_fake < 0.5).float()
        acc = torch.cat((pred_real, pred_fake)).sum().item()
        correct += acc
        total += batch_size * 2

        # Track losses
        epoch_g_loss += g_loss.item()
        epoch_d_loss += d_loss.item()

    # Compute average losses and accuracy
    avg_g_loss = epoch_g_loss / len(train_loader)
    avg_d_loss = epoch_d_loss / len(train_loader)
    accuracy = 100 * correct / total

    # Store metrics
    G_losses.append(avg_g_loss)
    D_losses.append(avg_d_loss)
    D_accuracies.append(accuracy)

    print(f"Epoch {epoch+1}/{EPOCHS} | G Loss: {avg_g_loss:.4f} | D Loss: {avg_d_loss:.4f} | D Acc: {accuracy:.2f}%")


# --------------------------------
# Save Generator weights
# --------------------------------
torch.save(netG.state_dict(), "generator_weights_ECGAN.pth")
print("Generator weights saved to 'generator_weights_ECGAN.pth'")


# Plotting of graphs after training

In [ ]:
# ================================
# Plot Losses and Accuracy for generator and discriminator after training
# ================================
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(G_losses, label="Generator Loss")
plt.plot(D_losses, label="Discriminator Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(D_accuracies, label="Discriminator Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.grid(True)

plt.show()

# Genarating images using trained generator weights

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import os
from torchvision import transforms
from PIL import Image

# Set device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters (must match training setup)
NZ = 128            # Noise vector size
CHANNELS = 3        # RGB image
NUM_CLASSES = 2     # 'Abnormal'=0, 'Normal'=1
IMAGE_SIZE = 224    # Output image dimensions

# Define the Generator architecture (must match trained model)
class Generator(nn.Module):
    def __init__(self, nz=128, channels=3, num_classes=2):
        super(Generator, self).__init__()
        self.nz = nz
        self.label_embedding = nn.Embedding(num_classes, nz)

        def block(in_feat, out_feat, k_size=4, stride=2, padding=1, dropout=False):
            layers = [
                nn.ConvTranspose2d(in_feat, out_feat, k_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_feat),
                nn.ReLU(True)
            ]
            if dropout:
                layers.append(nn.Dropout(0.3))
            return layers

        self.model = nn.Sequential(
            *block(nz * 2, 1024, 4, 1, 0),   # 1x1 → 4x4
            *block(1024, 512),              # 4x4 → 8x8
            *block(512, 256),               # 8x8 → 16x16
            *block(256, 128),               # 16x16 → 32x32
            *block(128, 64),                # 32x32 → 64x64
            *block(64, 32),                 # 64x64 → 128x128
            *block(32, 16),                 # 128x128 → 256x256
            nn.ConvTranspose2d(16, channels, 4, 2, 1),  # 256x256 → 512x512
            nn.Upsample(size=(224, 224)),              # Resize to 224×224
            nn.Tanh()                                   # Output in [-1, 1]
        )

    def forward(self, z, labels):
        embed = self.label_embedding(labels)
        x = torch.cat((z, embed), dim=1).view(-1, self.nz * 2, 1, 1)
        return self.model(x)

# === Load Pre-trained Weights from Kaggle Upload ===
weight_path = "/kaggle/input/weights/generator_weights_ECGAN.pth"

# Instantiate the generator and load weights
netG = Generator(nz=NZ, channels=CHANNELS, num_classes=NUM_CLASSES).to(device)
netG.load_state_dict(torch.load(weight_path, map_location=device))
netG.eval()  # Set to evaluation mode

# === Image Generation Settings ===
num_images = 383  # Number of images to generate
output_dir = "/kaggle/working/generated_normals_using_ECGAN"
os.makedirs(output_dir, exist_ok=True)

# Generate noise and Normal class labels (label = 1)
noise = torch.randn(num_images, NZ, device=device)
normal_labels = torch.full((num_images,), 1, dtype=torch.long, device=device)

# Generate fake images
with torch.no_grad():
    fake_images = netG(noise, normal_labels)  # Output in [-1, 1]
    fake_images = fake_images * 0.5 + 0.5     # Scale to [0, 1]

# Convert tensors to images and save
to_pil = transforms.ToPILImage()
for i in range(num_images):
    img = fake_images[i].cpu()
    img = to_pil(img)
    img.save(f"{output_dir}/normal_{i+1:03d}.png")

print(f"✅ Saved {num_images} 'Normal' images to: {output_dir}")


# Download the generated_normals_ECGAN as zip

In [ ]:
import shutil
from IPython.display import FileLink

# Zip the folder
shutil.make_archive('generated_normals_using_ECGAN', 'zip', 'generated_normals_using_ECGAN')

# Data Augmentation using CGAN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

# -------------Generator -------------------
class Generator(nn.Module):
    def __init__(self, nz=128, channels=3, num_classes=2):
        super(Generator, self).__init__()
        self.nz = nz
        self.label_embedding = nn.Embedding(num_classes, nz)

        def block(in_feat, out_feat, kernel_size=4, stride=2, padding=1):
            return [
                nn.ConvTranspose2d(in_feat, out_feat, kernel_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_feat),
                nn.ReLU(True)
            ]

        self.model = nn.Sequential(
            *block(nz * 2, 1024, 4, 1, 0),   # 1x1 → 4x4
            *block(1024, 512),              # 4x4 → 8x8
            *block(512, 256),              # 8x8 → 16x16
            *block(256, 128),              # 16x16 → 32x32
            *block(128, 64),               # 32x32 → 64x64
            *block(64, 32),                # 64x64 → 128x128
            *block(32, 16),                # 128x128 → 256x256
            nn.ConvTranspose2d(16, channels, 4, 2, 1),  # 256x256 → 512x512
            nn.Upsample(size=(224, 224)),              # Resize down to 224x224
            nn.Tanh()
        )

    def forward(self, noise, labels):
        label_emb = self.label_embedding(labels)
        x = torch.cat([noise, label_emb], dim=1).view(-1, self.nz * 2, 1, 1)
        return self.model(x)

#  ------------- Discriminator -------------
class Discriminator(nn.Module):
    def __init__(self, channels=3, num_classes=2, image_size=224):  # ✅ Use 224
        super(Discriminator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, image_size * image_size)  # ✅ 224x224 = 50176

        def block(in_feat, out_feat, kernel_size=4, stride=2, padding=1, bn=True):
            layers = [nn.Conv2d(in_feat, out_feat, kernel_size, stride, padding, bias=False)]
            if bn:
                layers.append(nn.BatchNorm2d(out_feat))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(channels + 1, 64, bn=False),  # 224 → 112
            *block(64, 128),                     # 112 → 56
            *block(128, 256),                    # 56 → 28
            *block(256, 512),                    # 28 → 14
            *block(512, 1024),                   # 14 → 7
            nn.Conv2d(1024, 1, 7, 1, 0)          # 7x7 → 1x1 output
        )

    def forward(self, imgs, labels):
        label_emb = self.label_embedding(labels).view(-1, 1, 224, 224)  # Reshape correctly
        x = torch.cat([imgs, label_emb], dim=1)
        out = self.model(x)
        return out.view(-1, 1)  # final logits


#  -------------Weight Initialization -------------
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

#  ------------- Hyperparameters  -------------
EPOCHS = 200
BATCH_SIZE = 32
LR = 0.0002
NZ = 128
CHANNELS = 3
NUM_CLASSES = 2
IMAGE_SIZE = 224
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------DataLoader  -------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,) * 3, (0.5,) * 3)
])

dataset = datasets.ImageFolder('/kaggle/input/dfuc-parta-resized', transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

#  ------------- Models, Loss, Optimizers  -------------
netG = Generator(nz=NZ, channels=CHANNELS, num_classes=NUM_CLASSES).to(device)
netD = Discriminator(channels=CHANNELS, num_classes=NUM_CLASSES, image_size=IMAGE_SIZE).to(device)
netG.apply(weights_init)
netD.apply(weights_init)

criterion = nn.BCEWithLogitsLoss()
optimizerG = optim.Adam(netG.parameters(), lr=LR, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=LR, betas=(0.5, 0.999))

#  ------------- Training Loop  -------------
os.makedirs("cgan_samples", exist_ok=True)
for epoch in range(EPOCHS):
    g_loss_total, d_loss_total, correct, total = 0, 0, 0, 0
    for imgs, labels in dataloader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.size(0)

        real = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)

        # Train Discriminator
        optimizerD.zero_grad()
        real_output = netD(imgs, labels)
        d_real_loss = criterion(real_output, real)

        noise = torch.randn(batch_size, NZ, device=device)
        fake_labels = torch.randint(0, NUM_CLASSES, (batch_size,), device=device)
        fake_imgs = netG(noise, fake_labels)

        fake_output = netD(fake_imgs.detach(), fake_labels)
        d_fake_loss = criterion(fake_output, fake)

        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        optimizerD.step()

        # Train Generator
        optimizerG.zero_grad()
        fake_output = netD(fake_imgs, fake_labels)
        g_loss = criterion(fake_output, real)
        g_loss.backward()
        optimizerG.step()

        # Accuracy
        pred_real = (real_output >= 0).float()
        pred_fake = (fake_output < 0).float()
        correct += pred_real.sum().item() + pred_fake.sum().item()
        total += 2 * batch_size

        g_loss_total += g_loss.item()
        d_loss_total += d_loss.item()

    avg_g_loss = g_loss_total / len(dataloader)
    avg_d_loss = d_loss_total / len(dataloader)
    d_acc = 100 * correct / total
    print(f"[Epoch {epoch+1}/{EPOCHS}] G Loss: {avg_g_loss:.4f} | D Loss: {avg_d_loss:.4f} | D Acc: {d_acc:.2f}%")

    # Save generated samples
    if (epoch + 1) % 10 == 0:
        netG.eval()
        with torch.no_grad():
            z = torch.randn(16, NZ, device=device)
            labels = torch.tensor([1]*16, device=device)
            samples = netG(z, labels)
            samples = (samples + 1) / 2
            save_image(samples, f"cgan_samples/epoch_{epoch+1:03d}.png", nrow=4)
        netG.train()

# -------------Save Model  -------------
torch.save(netG.state_dict(), "generator_weights_CGAN_stable.pth")
print("✅ Stable Generator weights saved.")


# Using generator_weights_CGAN

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
import os

# -------------Generator Definition  -------------
class Generator(nn.Module):
    def __init__(self, nz=128, channels=3, num_classes=2):
        super(Generator, self).__init__()
        self.nz = nz
        self.label_embedding = nn.Embedding(num_classes, nz)

        def block(in_feat, out_feat, kernel_size=4, stride=2, padding=1):
            return [
                nn.ConvTranspose2d(in_feat, out_feat, kernel_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_feat),
                nn.ReLU(True)
            ]

        self.model = nn.Sequential(
            *block(nz * 2, 1024, 4, 1, 0),   # 1x1 → 4x4
            *block(1024, 512),              # 4x4 → 8x8
            *block(512, 256),              # 8x8 → 16x16
            *block(256, 128),              # 16x16 → 32x32
            *block(128, 64),               # 32x32 → 64x64
            *block(64, 32),                # 64x64 → 128x128
            *block(32, 16),                # 128x128 → 256x256
            nn.ConvTranspose2d(16, channels, 4, 2, 1),  # 256x256 → 512x512
            nn.Upsample(size=(224, 224)),              # Resize down to 224x224
            nn.Tanh()
        )

    def forward(self, noise, labels):
        label_emb = self.label_embedding(labels)
        x = torch.cat([noise, label_emb], dim=1).view(-1, self.nz * 2, 1, 1)
        return self.model(x)

#  ------------- Parameters  -------------
NZ = 128
NUM_CLASSES = 2
IMAGE_SIZE = 224
CHANNELS = 3
NUM_IMAGES = 383  # Change as needed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  -------------Load Generator -------------
netG = Generator(nz=NZ, channels=CHANNELS, num_classes=NUM_CLASSES).to(device)
netG.load_state_dict(torch.load("/kaggle/input/weights-cgan/generator_weights_CGAN_stable.pth", map_location=device))
netG.eval()

#  ------------- Generate Noise & Labels  -------------
noise = torch.randn(NUM_IMAGES, NZ, device=device)
labels = torch.full((NUM_IMAGES,), 1, dtype=torch.long, device=device)  # class 1 = Normal

# -------------Generate Images -------------
with torch.no_grad():
    fake_images = netG(noise, labels)
    fake_images = (fake_images + 1) / 2  # Rescale from [-1, 1] to [0, 1]

#  ------------- Save Images  -------------
os.makedirs("generated_images_CGAN", exist_ok=True)
for i in range(NUM_IMAGES):
    save_image(fake_images[i], f"generated_images_CGAN/image_{i+1:03d}.png")

print(f"Saved {NUM_IMAGES} images to 'generated_images_CGAN/' folder.")


In [ ]:
import shutil
from IPython.display import FileLink

# Zip the folder
shutil.make_archive('generated_images_CGAN', 'zip', 'generated_images_CGAN')

# Plottingof Loss Curves

In [ ]:


# Plot Losses and Accuracy
 #-------------
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(G_losses, label="Generator Loss")
plt.plot(D_losses, label="Discriminator Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(D_accuracies, label="Discriminator Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.grid(True)

plt.show()

# Validation of generated images

In [ ]:
pip install torch torchvision piq pytorch-fid scipy

# FID Calculation between CGAN AND Normal

In [ ]:
import os
import torch
from torchvision import transforms
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset, DataLoader
from piq import FID
from torchvision.models import inception_v3

#  ------------- 1. Paths to real and generated image folders  -------------
real_folder = '/kaggle/input/dfuc-parta-resized/Normal'
gen_folder = '/kaggle/input/resized-dataset-cgan/resized_dataset_CGAN/Normal'

#  -------------2. Image Transform  -------------
transform = transforms.Compose([
    # transforms.Resize((299, 299)),  # Required size for InceptionV3
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Inception normalization
                         std=[0.229, 0.224, 0.225]),
])

# ========== 3. Custom Dataset ==========
class ImageFolderDataset(Dataset):
    def __init__(self, folder, transform):
        self.files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('png', 'jpg', 'jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = default_loader(self.files[idx])  # Loads image with PIL
        return self.transform(img)

# ========== 4. DataLoaders ==========
real_loader = DataLoader(ImageFolderDataset(real_folder, transform), batch_size=32)
gen_loader = DataLoader(ImageFolderDataset(gen_folder, transform), batch_size=32)

# ========== 5. Load InceptionV3 Model ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception = inception_v3(pretrained=True, aux_logits=True, transform_input=False)

inception.fc = torch.nn.Identity()  # Remove classification head
inception.eval()
inception = inception.to(device)

#  ------------- 6. Feature Extraction Function  -------------
def extract_features(loader):
    features = []
    with torch.no_grad():
        for imgs in loader:
            imgs = imgs.to(device)
            if imgs.size(1) == 1:
                imgs = imgs.repeat(1, 3, 1, 1)  # Convert grayscale to RGB
            feats = inception(imgs)
            features.append(feats.cpu())
    return torch.cat(features, dim=0)

# ------------- 7. Compute FID  -------------
print("Extracting features for real images...")
real_feats = extract_features(real_loader)
print("Extracting features for generated images...")
gen_feats = extract_features(gen_loader)

fid = FID()
fid_score = fid(gen_feats, real_feats)
print(f"\n✅ FID Score: {fid_score.item():.2f}")


# FID Calculation between Enhanced CGAN AND Normal

In [ ]:
import os
import torch
from torchvision import transforms
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset, DataLoader
from piq import FID
from torchvision.models import inception_v3

# ------------- 1. Paths to real and generated image folders  -------------
real_folder = '/kaggle/input/dfuc-parta-resized/Normal'
gen_folder = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN/Normal'

# ------------- 2. Image Transform  -------------
transform = transforms.Compose([
    # transforms.Resize((299, 299)),  # Required size for InceptionV3
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Inception normalization
                         std=[0.229, 0.224, 0.225]),
])

# ------------- 3. Custom Dataset -------------
class ImageFolderDataset(Dataset):
    def __init__(self, folder, transform):
        self.files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('png', 'jpg', 'jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = default_loader(self.files[idx])  # Loads image with PIL
        return self.transform(img)

#  ------------- 4. DataLoaders -------------
real_loader = DataLoader(ImageFolderDataset(real_folder, transform), batch_size=32)
gen_loader = DataLoader(ImageFolderDataset(gen_folder, transform), batch_size=32)

# ------------- 5. Load InceptionV3 Model  -------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception = inception_v3(pretrained=True, aux_logits=True, transform_input=False)

inception.fc = torch.nn.Identity()  # Remove classification head
inception.eval()
inception = inception.to(device)

#  ------------- 6. Feature Extraction Function  -------------
def extract_features(loader):
    features = []
    with torch.no_grad():
        for imgs in loader:
            imgs = imgs.to(device)
            if imgs.size(1) == 1:
                imgs = imgs.repeat(1, 3, 1, 1)  # Convert grayscale to RGB
            feats = inception(imgs)
            features.append(feats.cpu())
    return torch.cat(features, dim=0)

#  ------------- 7. Compute FID  -------------
print("Extracting features for real images...")
real_feats = extract_features(real_loader)
print("Extracting features for generated images...")
gen_feats = extract_features(gen_loader)

fid = FID()
fid_score = fid(gen_feats, real_feats)
print(f"\n✅ FID Score: {fid_score.item():.2f}")


# SSIM calculation between normal and generated normal images from CGAN

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset, DataLoader
import numpy as np
from skimage.metrics import structural_similarity as compare_ssim

real_folder = '/kaggle/input/dfuc-parta-resized/Normal'
gen_folder = '/kaggle/input/383-generated/383_generated'

#  Image transformation (resize and convert to tensor)
transform = transforms.Compose([
    transforms.ToTensor()
])

# 3. Dataset class to pair real and generated images
class PairedImageDataset(Dataset):
    def __init__(self, real_dir, gen_dir, transform=None):
        self.real_files = sorted([f for f in os.listdir(real_dir) if f.endswith(('jpg', 'png'))])
        self.gen_files = sorted([f for f in os.listdir(gen_dir) if f.endswith(('jpg', 'png'))])
        self.real_dir = real_dir
        self.gen_dir = gen_dir
        self.transform = transform

    def __len__(self):
        return min(len(self.real_files), len(self.gen_files))

    def __getitem__(self, idx):
        real_path = os.path.join(self.real_dir, self.real_files[idx])
        gen_path = os.path.join(self.gen_dir, self.gen_files[idx])

        real_img = default_loader(real_path)
        gen_img = default_loader(gen_path)

        if self.transform:
            real_img = self.transform(real_img)
            gen_img = self.transform(gen_img)

        return real_img, gen_img

# 4. Load paired images
dataset = PairedImageDataset(real_folder, gen_folder, transform)
loader = DataLoader(dataset, batch_size=1)

# 5. Compute SSIM for each image pair
ssim_scores = []

for real, gen in loader:
    real = real.squeeze().numpy()
    gen = gen.squeeze().numpy()

    # Convert from (C, H, W) to (H, W, C)
    real = np.transpose(real, (1, 2, 0)).clip(0, 1)
    gen = np.transpose(gen, (1, 2, 0)).clip(0, 1)

    # If grayscale, use single channel SSIM
    if real.shape[2] == 1 or gen.shape[2] == 1:
        score = compare_ssim(real[:, :, 0], gen[:, :, 0], data_range=1.0)
    else:
        score = compare_ssim(real, gen, channel_axis=2, data_range=1.0)

    ssim_scores.append(score)

# 6. Average SSIM
avg_ssim = np.mean(ssim_scores)
print(f"Average SSIM: {avg_ssim:.4f}")

# SSIM calculation between normal and generated normal images from Enhanced CGAN

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset, DataLoader
import numpy as np
from skimage.metrics import structural_similarity as compare_ssim

real_folder = '/kaggle/input/dfuc-parta-resized/Normal'
gen_folder = '/kaggle/input/383-generated-ecgan'

#  Image transformation (resize and convert to tensor)
transform = transforms.Compose([
    transforms.ToTensor()
])

# 3. Dataset class to pair real and generated images
class PairedImageDataset(Dataset):
    def __init__(self, real_dir, gen_dir, transform=None):
        self.real_files = sorted([f for f in os.listdir(real_dir) if f.endswith(('jpg', 'png'))])
        self.gen_files = sorted([f for f in os.listdir(gen_dir) if f.endswith(('jpg', 'png'))])
        self.real_dir = real_dir
        self.gen_dir = gen_dir
        self.transform = transform

    def __len__(self):
        return min(len(self.real_files), len(self.gen_files))

    def __getitem__(self, idx):
        real_path = os.path.join(self.real_dir, self.real_files[idx])
        gen_path = os.path.join(self.gen_dir, self.gen_files[idx])

        real_img = default_loader(real_path)
        gen_img = default_loader(gen_path)

        if self.transform:
            real_img = self.transform(real_img)
            gen_img = self.transform(gen_img)

        return real_img, gen_img

# 4. Load paired images
dataset = PairedImageDataset(real_folder, gen_folder, transform)
loader = DataLoader(dataset, batch_size=1)

# 5. Compute SSIM for each image pair
ssim_scores = []

for real, gen in loader:
    real = real.squeeze().numpy()
    gen = gen.squeeze().numpy()

    # Convert from (C, H, W) to (H, W, C)
    real = np.transpose(real, (1, 2, 0)).clip(0, 1)
    gen = np.transpose(gen, (1, 2, 0)).clip(0, 1)

    # If grayscale, use single channel SSIM
    if real.shape[2] == 1 or gen.shape[2] == 1:
        score = compare_ssim(real[:, :, 0], gen[:, :, 0], data_range=1.0)
    else:
        score = compare_ssim(real, gen, channel_axis=2, data_range=1.0)

    ssim_scores.append(score)

# 6. Average SSIM
avg_ssim = np.mean(ssim_scores)
print(f"Average SSIM: {avg_ssim:.4f}")

# PSNR calculation between normal and generated normal images from CGAN

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
from math import log10

# === 1. Folder paths ===
real_folder = '/kaggle/input/dfuc-parta-resized/Normal'
gen_folder = '/kaggle/input/383-generated/383_generated'

# === 2. Image transformation ===
transform = transforms.Compose([
    transforms.ToTensor()
])

# === 3. Dataset class to pair images ===
class PairedImageDataset(Dataset):
    def __init__(self, real_dir, gen_dir, transform=None):
        self.real_files = sorted([f for f in os.listdir(real_dir) if f.endswith(('jpg', 'png'))])
        self.gen_files = sorted([f for f in os.listdir(gen_dir) if f.endswith(('jpg', 'png'))])
        self.real_dir = real_dir
        self.gen_dir = gen_dir
        self.transform = transform

    def __len__(self):
        return min(len(self.real_files), len(self.gen_files))

    def __getitem__(self, idx):
        real_path = os.path.join(self.real_dir, self.real_files[idx])
        gen_path = os.path.join(self.gen_dir, self.gen_files[idx])

        real_img = default_loader(real_path)
        gen_img = default_loader(gen_path)

        if self.transform:
            real_img = self.transform(real_img)
            gen_img = self.transform(gen_img)

        return real_img, gen_img

# === 4. Load paired images ===
dataset = PairedImageDataset(real_folder, gen_folder, transform)
loader = DataLoader(dataset, batch_size=1)

# === 5. PSNR calculation function ===
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')  # Identical images
    return 20 * log10(1.0 / np.sqrt(mse))  # PSNR in dB

# === 6. Compute PSNR for each pair ===
psnr_scores = []

for real, gen in loader:
    real = real.squeeze().numpy()
    gen = gen.squeeze().numpy()

    # Convert to shape (H, W, C) and clip to [0, 1]
    real = np.transpose(real, (1, 2, 0)).clip(0, 1)
    gen = np.transpose(gen, (1, 2, 0)).clip(0, 1)

    # Compute PSNR
    psnr = calculate_psnr(real, gen)
    psnr_scores.append(psnr)

# === 7. Average PSNR ===
avg_psnr = np.mean(psnr_scores)
print(f"✅ Average PSNR: {avg_psnr:.2f} dB")

# PSNR calculation between normal and generated normal images from Enhanced CGAN

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
from math import log10

# === 1. Folder paths ===
real_folder = '/kaggle/input/dfuc-parta-resized/Normal'
gen_folder = '/kaggle/input/383-generated-ecgan'

# === 2. Image transformation ===
transform = transforms.Compose([
    transforms.ToTensor()
])

# === 3. Dataset class to pair images ===
class PairedImageDataset(Dataset):
    def __init__(self, real_dir, gen_dir, transform=None):
        self.real_files = sorted([f for f in os.listdir(real_dir) if f.endswith(('jpg', 'png'))])
        self.gen_files = sorted([f for f in os.listdir(gen_dir) if f.endswith(('jpg', 'png'))])
        self.real_dir = real_dir
        self.gen_dir = gen_dir
        self.transform = transform

    def __len__(self):
        return min(len(self.real_files), len(self.gen_files))

    def __getitem__(self, idx):
        real_path = os.path.join(self.real_dir, self.real_files[idx])
        gen_path = os.path.join(self.gen_dir, self.gen_files[idx])

        real_img = default_loader(real_path)
        gen_img = default_loader(gen_path)

        if self.transform:
            real_img = self.transform(real_img)
            gen_img = self.transform(gen_img)

        return real_img, gen_img

# === 4. Load paired images ===
dataset = PairedImageDataset(real_folder, gen_folder, transform)
loader = DataLoader(dataset, batch_size=1)

# === 5. PSNR calculation function ===
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')  # Identical images
    return 20 * log10(1.0 / np.sqrt(mse))  # PSNR in dB

# === 6. Compute PSNR for each pair ===
psnr_scores = []

for real, gen in loader:
    real = real.squeeze().numpy()
    gen = gen.squeeze().numpy()

    # Convert to shape (H, W, C) and clip to [0, 1]
    real = np.transpose(real, (1, 2, 0)).clip(0, 1)
    gen = np.transpose(gen, (1, 2, 0)).clip(0, 1)

    # Compute PSNR
    psnr = calculate_psnr(real, gen)
    psnr_scores.append(psnr)

# === 7. Average PSNR ===
avg_psnr = np.mean(psnr_scores)
print(f"✅ Average PSNR: {avg_psnr:.2f} dB")

# **Classification**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import callbacks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# **Data Loading**

In [ ]:
# Define Constants
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

BATCH_SIZE = 32
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
NUM_CLASSES = 2
EPOCHS = 200

# Data Pre-processing

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Define your dataset directory
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"  # Replace with the actual path

# Define preprocessing transformations
preprocess_transforms = transforms.Compose([
    transforms.ToTensor(),                       # Convert image to tensor
    transforms.Normalize((0.5, 0.5, 0.5),         # Normalize to [-1, 1]
                         (0.5, 0.5, 0.5))
])

# Apply transforms using ImageFolder
preprocessed_dataset = datasets.ImageFolder(root=data_dir, transform=preprocess_transforms)

# Create a DataLoader to view and use the data
loader = DataLoader(preprocessed_dataset, batch_size=32, shuffle=True)

# View class names
class_names = preprocessed_dataset.classes
print(f"Classes: {class_names}")


# Visualize 8 random images from a preprocessed PyTorch dataset

In [ ]:
# Function to display a tensor image using matplotlib
def imshow(img_tensor, title):
    # Convert the image tensor to a NumPy array and rearrange dimensions from (C, H, W) to (H, W, C)
    img = img_tensor.numpy().transpose((1, 2, 0))

    # De-normalize the image values from [-1, 1] range back to [0, 1] for correct display
    img = img * 0.5 + 0.5

    # Display the image using matplotlib
    plt.imshow(img)

    # Set the title (e.g., class name) above the image
    plt.title(title)

    # Hide the axes for a cleaner image display
    plt.axis('off')


# Select 8 random indices from the dataset for visualization
random_indices = random.sample(range(len(preprocessed_dataset)), 8)


# Create a new figure with specified size (12x6 inches)
plt.figure(figsize=(12, 6))

# Loop through the selected indices to plot each image
for i, idx in enumerate(random_indices):
    # Get the image and its corresponding label from the dataset
    image, label = preprocessed_dataset[idx]

    # Create a subplot at the appropriate position in a 2-row, 4-column grid
    plt.subplot(2, 4, i + 1)

    # Display the image and its class label using the imshow() function
    imshow(image, class_names[label])

# Adjust spacing between subplots to prevent overlapping
plt.tight_layout()

# Show the final figure with all subplots
plt.show()


# No. of classes

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import os

# Set paths
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"  # Adjust path if needed

# Image transformations
transform = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Normalize((0.5,)*3, (0.5,)*3)
])

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = full_dataset.classes  # ['Abnormal', 'Normal']
print(f"Classes: {class_names}")

# Train-test split (e.g., 70% train, 30% test)
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# **Data Cleaning**

In [ ]:
import os
from collections import Counter

# Path to your dataset
dataset_path = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Dictionary to hold extension counts per class
extension_counts = {}

# Loop through each class folder
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        extensions = []
        for file_name in os.listdir(class_path):
            if os.path.isfile(os.path.join(class_path, file_name)):
                ext = os.path.splitext(file_name)[-1].lower()
                extensions.append(ext)
        extension_counts[class_name] = Counter(extensions)

# Print the results
for class_name, counter in extension_counts.items():
    print(f"\nClass: {class_name}")
    for ext, count in counter.items():
        print(f"  {ext}: {count} files")


# BALANCING DATASET

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Define the directory path to the original dataset
TRAIN_PATH = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'

# Load the entire dataset without splitting
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    batch_size=32,  # Adjust based on your system's capability
    image_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    seed=0
)

# Count the occurrences of each class (0: Abnormal, 1: Normal)
total_counts = np.array([0, 0])  # [Count of 0s, Count of 1s]

# Iterate through the dataset to count labels
for _, labels in full_ds:
    unique, counts = np.unique(labels.numpy(), return_counts=True)
    for u, c in zip(unique, counts):
        total_counts[int(u)] += c

# Define class names
class_names = ["Abnormal", "Normal"]

# Plot the class distribution
colors = ['#d62728', '#1f77b4']  # Red for Abnormal, Blue for Normal
plt.figure(figsize=(8, 6))
plt.bar(class_names, total_counts, color=colors)
plt.xlabel("Class")
plt.ylabel("Number of Samples")
plt.title("Class Distribution after balancing Original Dataset")
plt.tight_layout()
plt.show()


# Print class counts
print(f"Abnormal (0): {total_counts[0]} samples")
print(f"Normal (1): {total_counts[1]} samples")


# **Visualization of Training Dataset**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
grid_size = 4  # Number of rows and columns in the grid

for images, labels in train_ds.take(1):  # Take one batch from the dataset
    for i in range(grid_size * grid_size):  # Display grid_size^2 images
        ax = plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

plt.tight_layout()  # Ensure no overlap between images and titles
plt.show()


# Splitting of Dataset (70:30)

In [ ]:
# Import necessary modules
import os                    
import shutil                
from torchvision import datasets  

# Path to the original dataset (already resized and organized in subfolders by class)
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Path to where train/test split data will be stored
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")   # Path for training data
test_dir = os.path.join(output_dir, "test")     # Path for testing data

# Load the full dataset using torchvision's ImageFolder
# This assumes the dataset has subfolders named after class labels
full_dataset = datasets.ImageFolder(root=data_dir)

# Get list of class names from the dataset (e.g., ['normal', 'abnormal'])
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create directory structure for the train and test folders
# Example: /kaggle/working/dataset_split/train/normal
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)  # Create subdirectories per class

# Split dataset into 70% training and 30% testing
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to their respective train/test folders
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:  # Use the saved indices from the random split
        path, label = full_dataset.samples[idx]  # Get original image path and its label
        class_name = class_names[label]          # Map label index to class name (e.g., 0 → 'normal')
        filename = os.path.basename(path)        # Extract the filename (e.g., 'image1.png')
        dest = os.path.join(split_folder, class_name, filename)  # Destination path for copy
        shutil.copy2(path, dest)  # Copy image file while preserving metadata

# Save the training images to train/ and test images to test/
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

# Print confirmation messages with final paths
print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


# Proposed CNN self-attention 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
import numpy as np
from tensorflow.keras import metrics

# Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim                      # Total embedding dimension
        self.num_heads = num_heads                      # Number of attention heads
        self.head_dim = embed_dim // num_heads          # Dimension per attention head
        
        # Ensure embedding dimension is divisible by number of heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        
        # Dense layers to project inputs into queries, keys, and values
        self.query_dense = layers.Dense(embed_dim)  
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        
        # Dense layer to combine multiple attention heads output
        self.combine_heads = layers.Dense(embed_dim)
        
    def attention(self, query, key, value):
        # Calculate scaled dot-product attention scores
        score = tf.matmul(query, key, transpose_b=True)   # Compute dot product of query and key^T
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32) # Dimensionality for scaling
        scaled_score = score / tf.math.sqrt(dim_key)      # Scale scores to stabilize gradients
        weights = tf.nn.softmax(scaled_score, axis=-1)    # Apply softmax to get attention weights
        output = tf.matmul(weights, value)                 # Weighted sum of values based on attention weights
        return output, weights
    
    def separate_heads(self, x, batch_size):
        # Reshape the input tensor to split into multiple attention heads
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim)) 
        # Transpose to shape: (batch_size, num_heads, sequence_length, head_dim)
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        
        # Flatten spatial dimensions (height and width) into one sequence dimension
        input_shape = tf.shape(inputs)
        height, width = input_shape[1], input_shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, input_shape[3]))  # shape: (batch, seq_len, channels)
        
        # Create queries, keys, and values projections
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        
        # Separate each into multiple heads
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        
        # Compute attention output and weights
        attention_output, attention_weights = self.attention(query, key, value)
        
        # Transpose back to (batch_size, seq_len, embed_dim)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        
        # Concatenate attention heads
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        
        # Final dense layer to combine heads into output
        output = self.combine_heads(concat_attention)
        
        # Reshape output back to spatial format (batch_size, height, width, embed_dim)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        
        return output

# Spatial Attention Block
#SpatialAttentionBlock is designed to perform spatial attention—that is, it learns where in the image the model should "pay more attention" by assigning weights to each spatial location
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        # 1x1 convolutions for query, key, and value feature generation
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)  # Final convolution after attention
        self.softmax = layers.Softmax(axis=-1)    # Softmax for attention weights
        
    def call(self, inputs):
        # Get dynamic shapes for batch size, height, width, channels
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        
        # Generate query, key, and value feature maps from input
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)
        
        # Reshape to (batch_size, sequence_length, filters) for matrix multiplication
        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))
        
        # Compute attention scores by dot product of query and key transpose
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))  # Scale scores
        
        # Softmax to obtain attention weights
        attention_weights = self.softmax(attention_scores)
        
        # Weighted sum of values based on attention weights
        attended_features = tf.matmul(attention_weights, value)
        
        # Reshape attended features back to spatial format
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        
        # Final convolution to refine features
        output = self.conv_out(attended_features)
        
        # Add residual connection if input channels match filter count
        if inputs.shape[-1] == self.filters:
            output = output + inputs
        
        return output

# Channel Attention Block
#In Channel Attention, the model learns to weigh the importance of each channel in a feature map.
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        
        # Global average and max pooling layers
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        
        # Dense layers to learn channel-wise attention weights
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')
        
    def call(self, inputs):
        # Average pooling branch: captures average channel information
        avg_pool = self.global_avg_pool(inputs)                     # Shape: (batch_size, channels)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)   # Reshape to (batch_size, 1, 1, channels)
        avg_pool = self.dense1(avg_pool)                             # Reduce dimension and non-linearity
        avg_pool = self.dense2(avg_pool)                             # Channel attention weights via sigmoid
        
        # Max pooling branch: captures prominent channel features
        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)
        
        # Combine attention weights from both branches
        attention = avg_pool + max_pool
        
        # Multiply input features by attention weights (channel-wise scaling)
        return inputs * attention

# ResNet-like Block with Attention
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_attention=True, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_attention = use_attention
        
        # Two convolutional layers with batch normalization
        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # Attention blocks (spatial and channel) if enabled
        if use_attention:
            self.spatial_attention = SpatialAttentionBlock(filters)
            self.channel_attention = ChannelAttentionBlock(filters)
        
        self.relu = layers.ReLU()
        
    def call(self, inputs):
        # First conv layer with activation and batch norm
        x = self.conv1(inputs)
        x = self.bn1(x)
        
        # Second conv layer with batch norm (no activation yet)
        x = self.conv2(x)
        x = self.bn2(x)
        
        # Apply attention blocks if enabled
        if self.use_attention:
            x = self.spatial_attention(x)
            x = self.channel_attention(x)
        
        # Residual skip connection if input and output have same channel dimension
        if inputs.shape[-1] == self.filters:
            x = x + inputs
        
        # Final activation after residual addition
        return self.relu(x)

# CNN Model with Self-Attention
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    """
    Build CNN model with self-attention mechanisms
    
    Args:
        IMAGE_WIDTH: Input image width
        IMAGE_HEIGHT: Input image height  
        NUM_CLASSES: Number of output classes
    
    Returns:
        Compiled Keras model
    """
    
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))  # Input layer with image shape
    
    # Normalize pixel values to [0,1]
    x = layers.Rescaling(1./255)(inputs)
    
    # Initial convolutional layer with large kernel and stride for downsampling
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Stage 1: Basic convolutions with batch norm
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Stage 2: Residual blocks with attention + downsampling
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    
    # Stage 3: Increased filter count and residual attention blocks + downsampling
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    
    # Stage 4: High-level features + self-attention + downsampling
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Apply self-attention over feature map
    x = SelfAttention(embed_dim=512, num_heads=8)(x)
    
    # Additional residual attention blocks after self-attention
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    
    # Global average pooling to reduce spatial dimensions
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    
    # Dropout for regularization to prevent overfitting
    x = layers.Dropout(0.5)(x)
    
    # Dense layer with ReLU activation for learning complex features
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer:
    # For binary classification, use sigmoid activation and binary crossentropy loss
    # For multi-class, use softmax activation and sparse categorical crossentropy
    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()
    
    # Create the Keras model instance
    model = Model(inputs, outputs, name="CNN_SelfAttention")
    
    # Compile model with Adam optimizer, selected loss, and common classification metrics
    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=[
            'accuracy',                            # Classification accuracy
            metrics.Precision(name='precision'), # Precision metric
            metrics.Recall(name='recall'),       # Recall metric
            metrics.AUC(name='auc')               # Area under ROC curve
        ]
    )

    return model



if __name__ == "__main__":
    # Define input image dimensions and number of classes
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    NUM_CLASSES = 2
    
    print("Building CNN with Self-Attention model...")
    
    # Build the model
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    
    # Print the model summary showing layers and parameters
    model.summary()
    
    # Generate a dummy input tensor (batch size 1, 224x224 RGB image)
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    
    # Run the model on dummy data to test forward pass
    output = model(dummy_input)
    
    # Print output tensor shape
    print(f"\nModel output shape: {output.shape}")
    
    # Print actual output values (probabilities or logits)
    print(f"Output value: {output.numpy()}")


In [ ]:
import tensorflow as tf  # Import TensorFlow library

# Paths to train and test directories (previously created and organized by class)
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters for image input size and batch processing
IMAGE_SIZE = (224, 224)   # Input image size (height, width) matching model requirement
BATCH_SIZE = 32           # Number of images per batch during training

# --- Load training dataset from directory ---
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,                  # Path to training images, organized in class subfolders
    label_mode='binary',        # Since there are only 2 classes (e.g., 'normal' and 'abnormal')
    image_size=IMAGE_SIZE,      # All images will be resized to this shape
    batch_size=BATCH_SIZE,      # How many images per batch
    shuffle=True                # Shuffle the dataset for better generalization during training
)

# --- Load test dataset from directory ---
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,                   # Path to testing images
    label_mode='binary',        # Binary classification (0 or 1)
    image_size=IMAGE_SIZE,      # Resize test images to same size as training
    batch_size=BATCH_SIZE,      # Process test data in batches
    shuffle=False               # Don't shuffle test data to maintain consistent evaluation
)


# Used to optimize the performance of your input pipeline in TensorFlow.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
inputs = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=200
)

# Plotting Accuracy and Loss from training

In [ ]:
import matplotlib.pyplot as plt  # Import matplotlib for plotting training graphs

# Create a figure with two subplots: one for accuracy, one for loss
plt.figure(figsize=(12, 5))  # Set the overall figure size (width=12, height=5 inches)

plt.subplot(1, 2, 1)  # Create first subplot (1 row, 2 columns, position 1)

# Plot accuracy over epochs from the training history
plt.plot(history.history['accuracy'], label='Accuracy')  # Plot training accuracy
plt.title('Model Accuracy')  # Title of the subplot
plt.xlabel('Epoch')          # X-axis label
plt.ylabel('Accuracy')       # Y-axis label
plt.legend()                 # Show legend (e.g., 'Val Accuracy')
plt.grid(True)               # Add grid for better readability


plt.subplot(1, 2, 2)  # Create second subplot (position 2)

# Plot loss over epochs from the training history
plt.plot(history.history['loss'], label='Loss')  # Plot training loss
plt.title('Model Loss')     # Title of the subplot
plt.xlabel('Epoch')         # X-axis label
plt.ylabel('Loss')          # Y-axis label
plt.legend()                # Show legend (e.g., 'Val Loss')
plt.grid(True)              # Add grid for better readability


plt.tight_layout()  # Automatically adjust subplot spacing for a clean layout
plt.show()          # Display the plot window


# Plotting val_accuracy and val_loss

In [ ]:
import matplotlib.pyplot as plt  # Import matplotlib for plotting training and validation graphs

# Create a figure with two subplots: one for validation accuracy, one for validation loss
plt.figure(figsize=(12, 5))  # Set the overall figure size (width=12, height=5 inches)

# -------- Validation Accuracy Plot --------
plt.subplot(1, 2, 1)  # Create first subplot (1 row, 2 columns, position 1)

plt.plot(history.history['val_accuracy'], label='Val Accuracy')  # Plot validation accuracy over epochs
plt.title('Validation Accuracy')  # Title of the subplot
plt.xlabel('Epoch')               # X-axis label
plt.ylabel('Accuracy')            # Y-axis label
plt.legend()                      # Show legend (e.g., 'Val Accuracy')
plt.grid(True)                    # Add grid lines for better readability

# -------- Validation Loss Plot --------
plt.subplot(1, 2, 2)  # Create second subplot (1 row, 2 columns, position 2)

plt.plot(history.history['val_loss'], label='Val Loss')  # Plot validation loss over epochs
plt.title('Validation Loss')      # Title of the subplot
plt.xlabel('Epoch')               # X-axis label
plt.ylabel('Loss')                # Y-axis label
plt.legend()                      # Show legend (e.g., 'Val Loss')
plt.grid(True)                    # Add grid lines for better readability

plt.tight_layout()  # Automatically adjust subplot spacing for a clean layout
plt.show()          # Display the combined figure with both plots


# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

# Get true labels from the validation dataset
y_true = np.concatenate([y.numpy() for x, y in validation_ds], axis=0)  # Ground truth labels

# Get predicted probabilities from the model
y_pred_probs = model.predict(validation_ds)  # Output will be probabilities between 0 and 1

# Convert probabilities to binary class predictions using threshold 0.5
y_pred = (y_pred_probs > 0.5).astype(int).flatten()  # Flatten in case output is shape (N, 1)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal (0)", "Abnormal (1)"])
disp.plot(cmap='Blues', values_format='d')  # 'd' ensures integer values are shown

plt.title("Confusion Matrix on Validation Set")
plt.grid(False)  # Disable grid for cleaner view
plt.show()


# ROC AUC

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Get ground truth labels from the validation dataset
y_true = np.concatenate([y.numpy() for x, y in validation_ds], axis=0)

# Step 2: Get predicted probabilities from the model
y_scores = model.predict(validation_ds).flatten()  # Flatten in case shape is (N, 1)

# Step 3: Compute False Positive Rate, True Positive Rate and thresholds
fpr, tpr, thresholds = roc_curve(y_true, y_scores)

# Step 4: Compute AUC
auc_score = roc_auc_score(y_true, y_scores)

# Step 5: Plot the ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_score:.4f})", color='blue')
plt.plot([0, 1], [0, 1], 'k--', label="Random Guessing")  # Diagonal line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


# Comparison with state-of-the-art  
1. LeNet
2. AlexNet
3. GoogleNet
4. EfficientNetB0
5. DenseNet121
6. ResNet101
7. InceptionV3
8. VGG16
9. Proposed Model

# EfficientnetB0

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, metrics
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

def build_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224, NUM_CLASSES=1):
    inputs = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    
    # Preprocessing layer
    x = layers.Lambda(preprocess_input)(inputs)

    # Load EfficientNetB0 base
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=x)
    base_model.trainable = False  # Freeze base for initial training

    # Classification head
    x = layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)  # Binary output

    # Create model
    model = tf.keras.Model(inputs, outputs, name="EfficientNetB0_Binary")

    # Compile with all important binary metrics
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc'),
            metrics.TruePositives(name='true_positives'),
            metrics.TrueNegatives(name='true_negatives'),
            metrics.FalsePositives(name='false_positives'),
            metrics.FalseNegatives(name='false_negatives')
        ]
    )

    model.summary()
    return model

In [ ]:
model1 = build_model()

In [ ]:
history1 = model1.fit(train_ds, validation_data = validation_ds,
                    epochs=200, verbose = 1)

In [ ]:
import matplotlib.pyplot as plt

def plot_accuracy_loss(history1):
    plt.figure(figsize=(12, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history1['accuracy'], label='Train Accuracy')
    plt.plot(history.history1['val_accuracy'], label='Val Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history1['loss'], label='Train Loss')
    plt.plot(history.history1['val_loss'], label='Val Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

def plot_confusion_matrix(model, validation_ds):
    y_true = np.concatenate([y.numpy() for _, y in validation_ds])
    y_pred_probs = model.predict(validation_ds).flatten()
    y_pred = (y_pred_probs > 0.5).astype(int)

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title("Confusion Matrix")
    plt.grid(False)
    plt.show()

    return cm


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt
# Get true labels from validation dataset
y_true = np.concatenate([y.numpy() for x, y in validation_ds], axis=0)

# Get predicted probabilities
y_pred_probs = model.predict(validation_ds)

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal (0)", "Abnormal (1)"])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix on Validation Set")
plt.grid(False)
plt.show()

# DenseNet121

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, metrics
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

def build_densenet_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224, NUM_CLASSES=1):
    inputs = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    
    # Preprocessing layer
    x = layers.Lambda(preprocess_input)(inputs)

    # Load DenseNet121 base
    base_model = DenseNet121(include_top=False, weights='imagenet', input_tensor=x)
    base_model.trainable = False  # Freeze base model

    # Classification head
    x = layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)  # Binary output

    # Create model
    model = tf.keras.Model(inputs, outputs, name="DenseNet121_Binary")

    # Compile with all important binary metrics
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc'),
            metrics.TruePositives(name='true_positives'),
            metrics.TrueNegatives(name='true_negatives'),
            metrics.FalsePositives(name='false_positives'),
            metrics.FalseNegatives(name='false_negatives')
        ]
    )

    model.summary()
    return model


In [ ]:
model2 = build_densenet_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224)

In [ ]:
history2 = model2.fit(train_ds, validation_data = validation_ds,
                    epochs=200, verbose = 1)

# ResNet101

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, metrics
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input

def build_resnet101_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224, NUM_CLASSES=1):
    inputs = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    
    # Preprocessing layer
    x = layers.Lambda(preprocess_input)(inputs)

    # Load ResNet101 base model
    base_model = ResNet101(include_top=False, weights='imagenet', input_tensor=x)
    base_model.trainable = False  # Freeze base initially

    # Classification head
    x = layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)  # Binary output

    # Create final model
    model = tf.keras.Model(inputs, outputs, name="ResNet101_Binary")

    # Compile with all important binary classification metrics
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc'),
            metrics.TruePositives(name='true_positives'),
            metrics.TrueNegatives(name='true_negatives'),
            metrics.FalsePositives(name='false_positives'),
            metrics.FalseNegatives(name='false_negatives')
        ]
    )

    model.summary()
    return model


In [ ]:
model3 = build_resnet101_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224)

In [ ]:
history3 = model3.fit(train_ds, validation_data = validation_ds,
                    epochs=200, verbose = 1)

# InceptionV3

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, metrics
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

def build_inceptionv3_model(INPUT_HEIGHT=224, INPUT_WIDTH=224, NUM_CLASSES=1):
    inputs = layers.Input(shape=(INPUT_HEIGHT, INPUT_WIDTH, 3))
    
    # Resize input images to 299x299 inside the model
    x = layers.Resizing(299, 299)(inputs)
    
    # Preprocessing for InceptionV3
    x = layers.Lambda(preprocess_input)(x)

    # Load InceptionV3 base model
    base_model = InceptionV3(include_top=False, weights='imagenet', input_tensor=x)
    base_model.trainable = False  # Freeze base model

    # Classification head
    x = layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="sigmoid", name="pred")(x)  # Binary output

    model = tf.keras.Model(inputs, outputs, name="InceptionV3_Binary")

    # Compile model with metrics for binary classification
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc'),
            metrics.TruePositives(name='true_positives'),
            metrics.TrueNegatives(name='true_negatives'),
            metrics.FalsePositives(name='false_positives'),
            metrics.FalseNegatives(name='false_negatives')
        ]
    )

    model.summary()
    return model


# Example usage:
model = build_inceptionv3_model(INPUT_HEIGHT=224, INPUT_WIDTH=224)


In [ ]:
model4 = build_inceptionv3_model()

In [ ]:
history4 = model4.fit(train_ds, validation_data = validation_ds,
                    epochs=200, verbose = 1)

# VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, metrics
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

def build_vgg16_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224, NUM_CLASSES=1):
    inputs = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

    # Preprocessing
    x = layers.Lambda(preprocess_input)(inputs)

    # Load VGG16 base model
    base_model = VGG16(include_top=False, weights='imagenet', input_tensor=x)
    base_model.trainable = False  # Freeze convolutional base

    # Classification head
    x = layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)  # Binary output

    # Build model
    model = tf.keras.Model(inputs, outputs, name="VGG16_Binary")

    # Compile with binary classification metrics
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc'),
            metrics.TruePositives(name='true_positives'),
            metrics.TrueNegatives(name='true_negatives'),
            metrics.FalsePositives(name='false_positives'),
            metrics.FalseNegatives(name='false_negatives')
        ]
    )

    model.summary()
    return model


In [ ]:
model5 = build_vgg16_model()

In [ ]:
history5 = model5.fit(train_ds, validation_data = validation_ds,
                    epochs=200, verbose = 1)

# LeNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, metrics

def build_lenet_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224, NUM_CLASSES=1):
    model = models.Sequential([
        layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),  # RGB images

        # First conv block
        layers.Conv2D(6, kernel_size=5, activation='relu', padding='same'),
        layers.AveragePooling2D(),

        # Second conv block
        layers.Conv2D(16, kernel_size=5, activation='relu'),
        layers.AveragePooling2D(),

        # Flatten + Dense layers
        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile model with all binary metrics
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc'),
            metrics.TruePositives(name='true_positives'),
            metrics.TrueNegatives(name='true_negatives'),
            metrics.FalsePositives(name='false_positives'),
            metrics.FalseNegatives(name='false_negatives')
        ]
    )

    model.summary()
    return model


In [ ]:
model6 = build_vgg16_model()

In [ ]:
history6 = model6.fit(train_ds, validation_data = validation_ds,
                    epochs=200, verbose = 1)

# Alexnet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, metrics

def build_alexnet_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224, NUM_CLASSES=1):
    model = models.Sequential([
        layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),

        # 1st Conv Layer
        layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=3, strides=2),

        # 2nd Conv Layer
        layers.Conv2D(filters=256, kernel_size=5, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=3, strides=2),

        # 3rd, 4th, 5th Conv Layers
        layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
        layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
        layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=3, strides=2),

        # Flatten and Fully Connected layers
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),

        # Output Layer for Binary Classification
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile with binary classification metrics
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc'),
            metrics.TruePositives(name='true_positives'),
            metrics.TrueNegatives(name='true_negatives'),
            metrics.FalsePositives(name='false_positives'),
            metrics.FalseNegatives(name='false_negatives')
        ]
    )

    model.summary()
    return model


In [ ]:
model7 = build_alexnet_model()

In [ ]:
history7 = model7.fit(train_ds, validation_data = validation_ds,
                    epochs=200, verbose = 1)

# GoogleNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, metrics, optimizers

def inception_module(x, f1, f3_in, f3_out, f5_in, f5_out, pool_proj):
    path1 = layers.Conv2D(f1, (1,1), padding='same', activation='relu')(x)

    path2 = layers.Conv2D(f3_in, (1,1), padding='same', activation='relu')(x)
    path2 = layers.Conv2D(f3_out, (3,3), padding='same', activation='relu')(path2)

    path3 = layers.Conv2D(f5_in, (1,1), padding='same', activation='relu')(x)
    path3 = layers.Conv2D(f5_out, (5,5), padding='same', activation='relu')(path3)

    path4 = layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(x)
    path4 = layers.Conv2D(pool_proj, (1,1), padding='same', activation='relu')(path4)

    return layers.concatenate([path1, path2, path3, path4], axis=-1)

def build_googlenet_model(IMAGE_HEIGHT=224, IMAGE_WIDTH=224, NUM_CLASSES=1):
    input_layer = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

    x = layers.Conv2D(64, (7,7), strides=(2,2), padding='same', activation='relu')(input_layer)
    x = layers.MaxPooling2D((3,3), strides=(2,2), padding='same')(x)
    x = layers.Conv2D(64, (1,1), activation='relu')(x)
    x = layers.Conv2D(192, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((3,3), strides=(2,2), padding='same')(x)

    # Inception modules
    x = inception_module(x, 64, 96, 128, 16, 32, 32)
    x = inception_module(x, 128, 128, 192, 32, 96, 64)
    x = layers.MaxPooling2D((3,3), strides=(2,2), padding='same')(x)

    x = inception_module(x, 192, 96, 208, 16, 48, 64)
    x = inception_module(x, 160, 112, 224, 24, 64, 64)
    x = inception_module(x, 128, 128, 256, 24, 64, 64)
    x = inception_module(x, 112, 144, 288, 32, 64, 64)
    x = inception_module(x, 256, 160, 320, 32, 128, 128)
    x = layers.MaxPooling2D((3,3), strides=(2,2), padding='same')(x)

    x = inception_module(x, 256, 160, 320, 32, 128, 128)
    x = inception_module(x, 384, 192, 384, 48, 128, 128)

    # Final layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=input_layer, outputs=output)

    # Compile
    model.compile(
        optimizer=optimizers.Adam(1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc'),
            metrics.TruePositives(name='true_positives'),
            metrics.TrueNegatives(name='true_negatives'),
            metrics.FalsePositives(name='false_positives'),
            metrics.FalseNegatives(name='false_negatives')
        ]
    )

    model.summary()
    return model


In [ ]:
model8 = build_googlenet_model()

In [ ]:
history8 = model8.fit(train_ds, validation_data = validation_ds,
                    epochs=100, verbose = 1)

# Proposed Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, losses
import numpy as np
from tensorflow.keras import metrics

# Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim                      # Total embedding dimension
        self.num_heads = num_heads                      # Number of attention heads
        self.head_dim = embed_dim // num_heads          # Dimension per attention head
        
        # Ensure embedding dimension is divisible by number of heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        
        # Dense layers to project inputs into queries, keys, and values
        self.query_dense = layers.Dense(embed_dim)  
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        
        # Dense layer to combine multiple attention heads output
        self.combine_heads = layers.Dense(embed_dim)
        
    def attention(self, query, key, value):
        # Calculate scaled dot-product attention scores
        score = tf.matmul(query, key, transpose_b=True)   # Compute dot product of query and key^T
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32) # Dimensionality for scaling
        scaled_score = score / tf.math.sqrt(dim_key)      # Scale scores to stabilize gradients
        weights = tf.nn.softmax(scaled_score, axis=-1)    # Apply softmax to get attention weights
        output = tf.matmul(weights, value)                 # Weighted sum of values based on attention weights
        return output, weights
    
    def separate_heads(self, x, batch_size):
        # Reshape the input tensor to split into multiple attention heads
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim)) 
        # Transpose to shape: (batch_size, num_heads, sequence_length, head_dim)
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        
        # Flatten spatial dimensions (height and width) into one sequence dimension
        input_shape = tf.shape(inputs)
        height, width = input_shape[1], input_shape[2]
        x = tf.reshape(inputs, (batch_size, height * width, input_shape[3]))  # shape: (batch, seq_len, channels)
        
        # Create queries, keys, and values projections
        query = self.query_dense(x)
        key = self.key_dense(x)
        value = self.value_dense(x)
        
        # Separate each into multiple heads
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        
        # Compute attention output and weights
        attention_output, attention_weights = self.attention(query, key, value)
        
        # Transpose back to (batch_size, seq_len, embed_dim)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        
        # Concatenate attention heads
        concat_attention = tf.reshape(attention_output, (batch_size, height * width, self.embed_dim))
        
        # Final dense layer to combine heads into output
        output = self.combine_heads(concat_attention)
        
        # Reshape output back to spatial format (batch_size, height, width, embed_dim)
        output = tf.reshape(output, (batch_size, height, width, self.embed_dim))
        
        return output

# Spatial Attention Block
#SpatialAttentionBlock is designed to perform spatial attention—that is, it learns where in the image the model should "pay more attention" by assigning weights to each spatial location
class SpatialAttentionBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(SpatialAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        # 1x1 convolutions for query, key, and value feature generation
        self.conv1 = layers.Conv2D(filters, 1, activation='relu')
        self.conv2 = layers.Conv2D(filters, 1, activation='relu')
        self.conv3 = layers.Conv2D(filters, 1, activation='relu')
        self.conv_out = layers.Conv2D(filters, 1)  # Final convolution after attention
        self.softmax = layers.Softmax(axis=-1)    # Softmax for attention weights
        
    def call(self, inputs):
        # Get dynamic shapes for batch size, height, width, channels
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        
        # Generate query, key, and value feature maps from input
        query = self.conv1(inputs)
        key = self.conv2(inputs)
        value = self.conv3(inputs)
        
        # Reshape to (batch_size, sequence_length, filters) for matrix multiplication
        query = tf.reshape(query, (batch_size, height * width, self.filters))
        key = tf.reshape(key, (batch_size, height * width, self.filters))
        value = tf.reshape(value, (batch_size, height * width, self.filters))
        
        # Compute attention scores by dot product of query and key transpose
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.filters, tf.float32))  # Scale scores
        
        # Softmax to obtain attention weights
        attention_weights = self.softmax(attention_scores)
        
        # Weighted sum of values based on attention weights
        attended_features = tf.matmul(attention_weights, value)
        
        # Reshape attended features back to spatial format
        attended_features = tf.reshape(attended_features, (batch_size, height, width, self.filters))
        
        # Final convolution to refine features
        output = self.conv_out(attended_features)
        
        # Add residual connection if input channels match filter count
        if inputs.shape[-1] == self.filters:
            output = output + inputs
        
        return output

# Channel Attention Block
#In Channel Attention, the model learns to weigh the importance of each channel in a feature map.
class ChannelAttentionBlock(layers.Layer):
    def __init__(self, filters, reduction_ratio=16, **kwargs):
        super(ChannelAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction_ratio = reduction_ratio
        
        # Global average and max pooling layers
        self.global_avg_pool = layers.GlobalAveragePooling2D()
        self.global_max_pool = layers.GlobalMaxPooling2D()
        
        # Dense layers to learn channel-wise attention weights
        self.dense1 = layers.Dense(filters // reduction_ratio, activation='relu')
        self.dense2 = layers.Dense(filters, activation='sigmoid')
        
    def call(self, inputs):
        # Average pooling branch: captures average channel information
        avg_pool = self.global_avg_pool(inputs)                     # Shape: (batch_size, channels)
        avg_pool = tf.expand_dims(tf.expand_dims(avg_pool, 1), 1)   # Reshape to (batch_size, 1, 1, channels)
        avg_pool = self.dense1(avg_pool)                             # Reduce dimension and non-linearity
        avg_pool = self.dense2(avg_pool)                             # Channel attention weights via sigmoid
        
        # Max pooling branch: captures prominent channel features
        max_pool = self.global_max_pool(inputs)
        max_pool = tf.expand_dims(tf.expand_dims(max_pool, 1), 1)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)
        
        # Combine attention weights from both branches
        attention = avg_pool + max_pool
        
        # Multiply input features by attention weights (channel-wise scaling)
        return inputs * attention

# ResNet-like Block with Attention
class ResidualAttentionBlock(layers.Layer):
    def __init__(self, filters, use_attention=True, **kwargs):
        super(ResidualAttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        self.use_attention = use_attention
        
        # Two convolutional layers with batch normalization
        self.conv1 = layers.Conv2D(filters, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # Attention blocks (spatial and channel) if enabled
        if use_attention:
            self.spatial_attention = SpatialAttentionBlock(filters)
            self.channel_attention = ChannelAttentionBlock(filters)
        
        self.relu = layers.ReLU()
        
    def call(self, inputs):
        # First conv layer with activation and batch norm
        x = self.conv1(inputs)
        x = self.bn1(x)
        
        # Second conv layer with batch norm (no activation yet)
        x = self.conv2(x)
        x = self.bn2(x)
        
        # Apply attention blocks if enabled
        if self.use_attention:
            x = self.spatial_attention(x)
            x = self.channel_attention(x)
        
        # Residual skip connection if input and output have same channel dimension
        if inputs.shape[-1] == self.filters:
            x = x + inputs
        
        # Final activation after residual addition
        return self.relu(x)

# CNN Model with Self-Attention
def build_cnn_attention_model(IMAGE_WIDTH=224, IMAGE_HEIGHT=224, NUM_CLASSES=2):
    """
    Build CNN model with self-attention mechanisms
    
    Args:
        IMAGE_WIDTH: Input image width
        IMAGE_HEIGHT: Input image height  
        NUM_CLASSES: Number of output classes
    
    Returns:
        Compiled Keras model
    """
    
    inputs = layers.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))  # Input layer with image shape
    
    # Normalize pixel values to [0,1]
    x = layers.Rescaling(1./255)(inputs)
    
    # Initial convolutional layer with large kernel and stride for downsampling
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Stage 1: Basic convolutions with batch norm
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Stage 2: Residual blocks with attention + downsampling
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    x = ResidualAttentionBlock(128, use_attention=True)(x)
    
    # Stage 3: Increased filter count and residual attention blocks + downsampling
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    x = ResidualAttentionBlock(256, use_attention=True)(x)
    
    # Stage 4: High-level features + self-attention + downsampling
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = layers.MaxPooling2D(2)(x)
    
    # Apply self-attention over feature map
    x = SelfAttention(embed_dim=512, num_heads=8)(x)
    
    # Additional residual attention blocks after self-attention
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    x = ResidualAttentionBlock(512, use_attention=True)(x)
    
    # Global average pooling to reduce spatial dimensions
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    
    # Dropout for regularization to prevent overfitting
    x = layers.Dropout(0.5)(x)
    
    # Dense layer with ReLU activation for learning complex features
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer:
    # For binary classification, use sigmoid activation and binary crossentropy loss
    # For multi-class, use softmax activation and sparse categorical crossentropy
    if NUM_CLASSES == 2:
        outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)
        loss_fn = losses.BinaryCrossentropy()
    else:
        outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
        loss_fn = losses.SparseCategoricalCrossentropy()
    
    # Create the Keras model instance
    model = Model(inputs, outputs, name="CNN_SelfAttention")
    
    # Compile model with Adam optimizer, selected loss, and common classification metrics
    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=[
            'accuracy',                            # Classification accuracy
            metrics.Precision(name='precision'), # Precision metric
            metrics.Recall(name='recall'),       # Recall metric
            metrics.AUC(name='auc')               # Area under ROC curve
        ]
    )

    return model


# Usage example
if __name__ == "__main__":
    # Define input image dimensions and number of classes
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    NUM_CLASSES = 2
    
    print("Building CNN with Self-Attention model...")
    
    # Build the model
    model = build_cnn_attention_model(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES)
    
    # Print the model summary showing layers and parameters
    model.summary()
    
    # Generate a dummy input tensor (batch size 1, 224x224 RGB image)
    dummy_input = tf.random.normal((1, IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    
    # Run the model on dummy data to test forward pass
    output = model(dummy_input)
    
    # Print output tensor shape
    print(f"\nModel output shape: {output.shape}")
    
    # Print actual output values (probabilities or logits)
    print(f"Output value: {output.numpy()}")


In [ ]:
import os
import shutil
from torchvision import datasets
from torch.utils.data import random_split

# Original dataset root folder
data_dir = "/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN"

# Where to save split datasets
output_dir = "/kaggle/working/dataset_split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Create folder structure for train/test split
for split_dir in [train_dir, test_dir]:
    for cls in class_names:
        os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

# Split dataset indices
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Function to copy images to target folder based on split
def save_split_dataset(dataset, split_folder):
    for idx in dataset.indices:
        path, label = full_dataset.samples[idx]
        class_name = class_names[label]
        filename = os.path.basename(path)
        dest = os.path.join(split_folder, class_name, filename)
        shutil.copy2(path, dest)

# Save train and test splits
save_split_dataset(train_dataset, train_dir)
save_split_dataset(test_dataset, test_dir)

print("Datasets saved in folder structure:")
print(f"Train: {train_dir}")
print(f"Test: {test_dir}")


In [ ]:
import tensorflow as tf

# Paths to train and test directories
train_dir = "/kaggle/working/dataset_split/train"
test_dir = "/kaggle/working/dataset_split/test"

# Parameters
IMAGE_SIZE = (224,224)   # Match your model input size
BATCH_SIZE = 32

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='binary',        # since you have 2 classes: Abnormal and Normal
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='binary',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import layers, models, optimizers, losses, metrics
from tensorflow.keras.applications import EfficientNetB0

# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 1
DATA_DIR = '/kaggle/input/resized-dataset-ecgan/resized_dataset_ECGAN'  # Replace if needed

# Step 1: Create dataframe with file paths and labels
image_paths = []
labels = []
for label, subdir in enumerate(sorted(os.listdir(DATA_DIR))):
    folder_path = os.path.join(DATA_DIR, subdir)
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(folder_path, fname))
            labels.append(label)

df = pd.DataFrame({'filepath': image_paths, 'label': labels})
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle

# Define the model
def build_model(image_height=224, image_width=224, num_classes=1):
    inputs = layers.Input(shape=(image_height, image_width, 3))
    x = tf.keras.applications.efficientnet.preprocess_input(inputs)
    
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=x)
    base_model.trainable = False  # Freeze base
    
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)  # sigmoid for binary

    model = models.Model(inputs, outputs)

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss=losses.BinaryCrossentropy(),
        metrics=[
            "accuracy",
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall'),
            metrics.AUC(name='auc')
        ]
    )
    return model

# Step 2: Apply Stratified K-Fold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(df['filepath'], df['label'])):
    print(f"\n========== Fold {fold+1} ==========")

    train_files = df.iloc[train_idx]
    val_files = df.iloc[val_idx]

    # Function to load dataset from DataFrame
    def load_dataset(file_df):
        paths = file_df['filepath'].values
        labels = file_df['label'].values
        ds = tf.data.Dataset.from_tensor_slices((paths, labels))

        def process(path, label):
            img = tf.io.read_file(path)
            img = tf.image.decode_jpeg(img, channels=3)
            img = tf.image.resize(img, IMAGE_SIZE)
            img = tf.keras.applications.efficientnet.preprocess_input(img)
            return img, tf.cast(label, tf.float32)

        ds = ds.map(process).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        return ds

    train_ds = load_dataset(train_files)
    val_ds = load_dataset(val_files)

    # Step 3: Build and train model
    model = build_model()
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, verbose=1)

    # Optional: Evaluate on validation
    print("\nValidation Metrics:")
    model.evaluate(val_ds, verbose=1)


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100
)